In [1]:
import os
import sys
import json

#os.environ['CUDA_VISIBLE_DEVICES']='1'
sys.path.append('..')

import config

In [2]:
from utils.dataset import ImageNetDataset
from utils.transform import train_transform, valid_transform
from utils.generator import generator
from utils.callbacks import get_callbacks

Using TensorFlow backend.


# Prepare data

In [3]:
# labels mapping
with open(config.MAP_CLS) as f:
    label_to_class = json.load(f)  
folder_to_label = {v[0]: k for k, v in label_to_class.items()}

In [4]:
# prepare datasets
train_dataset = ImageNetDataset(config.TRAIN_DIR, folder_to_label, transform=train_transform)
valid_dataset = ImageNetDataset(config.VALID_DIR, folder_to_label, transform=valid_transform)

In [5]:
# prepare generators 
train_gen = generator(train_dataset, batch_size=256, num_workers=12, shuffle=True)
valid_gen = generator(valid_dataset, batch_size=1, num_workers=12, shuffle=True)

# Build Model

In [6]:
from nn_models.dpn import DPN92
from nn_models.se_models.se_resnet import SEResNet18, SEResNet34
from keras.optimizers import SGD
from keras.utils import multi_gpu_model

In [7]:
# prepare model

model = SEResNet34(input_shape=(224, 224, 3))

In [8]:
model = multi_gpu_model(model, gpus=2)

In [9]:
#model.load_weights('../../checkpoints/se_resnet18/weights_ep210.h5')

In [10]:
opt = SGD(lr=0.6, momentum=0.9)
model.compile(opt, 'categorical_crossentropy', 
              ['categorical_accuracy', 'top_k_categorical_accuracy'])

In [11]:
callbacks = get_callbacks('se_resnet34', 
                          checkpoints_dir=config.CHECKPOINTS_DIR, 
                          monitor='val_loss',
                          log_dir=config.LOGS_DIR)

Instructions for updating:
Use the retry module or similar alternatives.


# Train

In [21]:
import keras.backend as K

K.set_value(model.optimizer.lr, 0.0005)

In [13]:
new_callbacks = callbacks[:-1]

In [14]:
model.fit_generator(train_gen,
                    steps_per_epoch=5000, 
                    initial_epoch=0,
                    epochs=200, 
                    validation_data=valid_gen, 
                    validation_steps=10000, 
                    callbacks=callbacks)

Epoch 1/200

Epoch 00001: LearningRateScheduler reducing learning rate to 0.6000000238418579.
5000/5000 [==============================] - 2603s 521ms/step - loss: 5.2210 - categorical_accuracy: 0.1157 - top_k_categorical_accuracy: 0.2685 - val_loss: 4.5846 - val_categorical_accuracy: 0.1897 - val_top_k_categorical_accuracy: 0.4023

Epoch 00001: val_loss improved from inf to 4.58462, saving model to /home/user/project/checkpoints/se_resnet34/weights-0.1897.h5
Epoch 2/200

Epoch 00002: LearningRateScheduler reducing learning rate to 0.6000000238418579.
5000/5000 [==============================] - 2590s 518ms/step - loss: 4.0190 - categorical_accuracy: 0.2666 - top_k_categorical_accuracy: 0.5115 - val_loss: 4.0515 - val_categorical_accuracy: 0.2635 - val_top_k_categorical_accuracy: 0.5176

Epoch 00002: val_loss improved from 4.58462 to 4.05150, saving model to /home/user/project/checkpoints/se_resnet34/weights-0.2635.h5
Epoch 3/200

Epoch 00003: LearningRateScheduler reducing learning ra

5000/5000 [==============================] - 2581s 516ms/step - loss: 2.4105 - categorical_accuracy: 0.5437 - top_k_categorical_accuracy: 0.7848 - val_loss: 2.7403 - val_categorical_accuracy: 0.4886 - val_top_k_categorical_accuracy: 0.7411

Epoch 00020: val_loss did not improve
Epoch 21/200

Epoch 00021: LearningRateScheduler reducing learning rate to 0.6000000238418579.
5000/5000 [==============================] - 2580s 516ms/step - loss: 2.3920 - categorical_accuracy: 0.5472 - top_k_categorical_accuracy: 0.7875 - val_loss: 2.5978 - val_categorical_accuracy: 0.5154 - val_top_k_categorical_accuracy: 0.7599

Epoch 00021: val_loss did not improve
Epoch 22/200

Epoch 00022: LearningRateScheduler reducing learning rate to 0.6000000238418579.
5000/5000 [==============================] - 2581s 516ms/step - loss: 2.3694 - categorical_accuracy: 0.5514 - top_k_categorical_accuracy: 0.7907 - val_loss: 2.4949 - val_categorical_accuracy: 0.5277 - val_top_k_categorical_accuracy: 0.7765

Epoch 00022


Epoch 00041: val_loss improved from 2.35500 to 1.65993, saving model to /home/user/project/checkpoints/se_resnet34/weights-0.6543.h5
Epoch 42/200

Epoch 00042: LearningRateScheduler reducing learning rate to 0.06000000238418579.
5000/5000 [==============================] - 2585s 517ms/step - loss: 1.5076 - categorical_accuracy: 0.6796 - top_k_categorical_accuracy: 0.8727 - val_loss: 1.6858 - val_categorical_accuracy: 0.6405 - val_top_k_categorical_accuracy: 0.8532

Epoch 00042: val_loss did not improve
Epoch 43/200

Epoch 00043: LearningRateScheduler reducing learning rate to 0.06000000238418579.
5000/5000 [==============================] - 2586s 517ms/step - loss: 1.4809 - categorical_accuracy: 0.6805 - top_k_categorical_accuracy: 0.8738 - val_loss: 1.6562 - val_categorical_accuracy: 0.6412 - val_top_k_categorical_accuracy: 0.8573

Epoch 00043: val_loss improved from 1.65993 to 1.65623, saving model to /home/user/project/checkpoints/se_resnet34/weights-0.6412.h5
Epoch 44/200

Epoch 0


Epoch 00062: val_loss did not improve
Epoch 63/200

Epoch 00063: LearningRateScheduler reducing learning rate to 0.06000000238418579.
5000/5000 [==============================] - 2588s 518ms/step - loss: 1.3923 - categorical_accuracy: 0.6952 - top_k_categorical_accuracy: 0.8841 - val_loss: 1.7633 - val_categorical_accuracy: 0.6232 - val_top_k_categorical_accuracy: 0.8418

Epoch 00063: val_loss did not improve
Epoch 64/200

Epoch 00064: LearningRateScheduler reducing learning rate to 0.06000000238418579.
5000/5000 [==============================] - 2587s 517ms/step - loss: 1.3890 - categorical_accuracy: 0.6962 - top_k_categorical_accuracy: 0.8841 - val_loss: 1.7102 - val_categorical_accuracy: 0.6371 - val_top_k_categorical_accuracy: 0.8481

Epoch 00064: val_loss did not improve
Epoch 65/200

Epoch 00065: LearningRateScheduler reducing learning rate to 0.06000000238418579.
5000/5000 [==============================] - 2589s 518ms/step - loss: 1.3899 - categorical_accuracy: 0.6954 - top_k

Epoch 84/200

Epoch 00084: LearningRateScheduler reducing learning rate to 0.006000000052154064.
5000/5000 [==============================] - 2581s 516ms/step - loss: 1.2110 - categorical_accuracy: 0.7301 - top_k_categorical_accuracy: 0.9016 - val_loss: 1.4844 - val_categorical_accuracy: 0.6728 - val_top_k_categorical_accuracy: 0.8759

Epoch 00084: val_loss did not improve
Epoch 85/200

Epoch 00085: LearningRateScheduler reducing learning rate to 0.006000000052154064.
5000/5000 [==============================] - 2582s 516ms/step - loss: 1.2040 - categorical_accuracy: 0.7304 - top_k_categorical_accuracy: 0.9018 - val_loss: 1.4722 - val_categorical_accuracy: 0.6804 - val_top_k_categorical_accuracy: 0.8764

Epoch 00085: val_loss did not improve
Epoch 86/200

Epoch 00086: LearningRateScheduler reducing learning rate to 0.006000000052154064.
5000/5000 [==============================] - 2581s 516ms/step - loss: 1.2020 - categorical_accuracy: 0.7299 - top_k_categorical_accuracy: 0.9020 - val_

...

Process Process-1927:
Process Process-1924:
Process Process-1873:
Process Process-1929:
Process Process-1930:
Process Process-1880:
Process Process-1928:
Process Process-1878:
Process Process-1921:
Process Process-1882:
Process Process-1884:
Process Process-1925:
Process Process-1879:
Process Process-1922:
Process Process-1877:
Process Process-1883:
Process Process-1926:
Process Process-1932:
Traceback (most recent call last):
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-1875:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-1874:
Traceback (most recent call last):
Traceback (most recent call l

  File "/opt/anaconda/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/opt/anaconda/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **s

  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/opt/anaconda/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/opt/anaconda/anaconda3/lib/python3.6/selectors.py",

KeyboardInterrupt
KeyboardInterrupt
  File "/opt/anaconda/anaconda3/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/opt/anaconda/anaconda3/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
  File "/opt/anaconda/anaconda3/lib/python3.6/site-packages/albumentations/core/composition.py", line 58, in __call__
    data = self.run_transforms_if_needed(need_to_run, data)
KeyboardInterrupt
  File "/opt/anaconda/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
KeyboardInterrupt
  File "/opt/anaconda/anaconda3/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/opt/anaconda/anaconda3/lib/python3.6/sit

KeyboardInterrupt: 

In [23]:
model.save_weights('../../checkpoints/se_resnet18/weights_ep310.h5')